For hyper parameter tuning we sholud import scikeras library

In [36]:
import pandas as pd  
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping

In [37]:
df=pd.read_csv("Churn_Modelling.csv")

In [38]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [39]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [40]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [41]:
label_encoder_gender=LabelEncoder()
df["Gender"]=label_encoder_gender.fit_transform(df["Gender"])

In [42]:
one_hot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=one_hot_encoder_geo.fit_transform(df[["Geography"]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_encoder_geo.get_feature_names_out(["Geography"]))

In [43]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [44]:
df=pd.concat([df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [45]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


Independent and Dependent features

In [46]:
X=df.drop("Exited",axis=1)
y=df["Exited"]

In [47]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

standardization

In [48]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

saving into pickle 

In [49]:
import pickle

In [50]:
with open("Label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("Onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(one_hot_encoder_geo,file)

with open("Scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

In [51]:
## Define a function to create the model and try different parameters(keras Classifier)
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    #for remaining hidden layers
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])

    return model 

Create a Keras Classifier

In [55]:
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [56]:
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2,3],
    #'batch_size':[10,20],
    'epochs':[50,100]
}

GRID SEARCH CV

In [57]:
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)

grid_result=grid.fit(X_train,y_train)

#print the best parameters
print("Best Parameters: ",grid_result.best_params_)
# print the best score
print("Best Score: ",grid_result.best_score_)

d:\Deep Learning Project Using ANN\venv\Lib\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(
d:\Deep Learning Project Using ANN\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/50


250/250 [==============================] - 2s 3ms/step - loss: 2.9750 - accuracy: 0.0214
Epoch 2/50
250/250 [==============================] - 1s 2ms/step - loss: 2.6617 - accuracy: 0.0226
Epoch 3/50
250/250 [==============================] - 1s 2ms/step - loss: 2.4740 - accuracy: 0.0243
Epoch 4/50
250/250 [==============================] - 1s 2ms/step - loss: 2.3438 - accuracy: 0.0294
Epoch 5/50
250/250 [==============================] - 1s 2ms/step - loss: 2.2215 - accuracy: 0.0364
Epoch 6/50
250/250 [==============================] - 1s 3ms/step - loss: 2.1316 - accuracy: 0.0439
Epoch 7/50
250/250 [==============================] - 1s 3ms/step - loss: 2.0492 - accuracy: 0.0468
Epoch 8/50
250/250 [==============================] - 1s 2ms/step - loss: 1.9694 - accuracy: 0.0542
Epoch 9/50
250/250 [==============================] - 1s 2ms/step - loss: 1.9015 - accuracy: 0.0558
Epoch 10/50
250/250 [==============================] - 1s 2ms/step - loss: 1.8336 - accuracy: 0.

In [60]:
grid_result.score

<bound method BaseSearchCV.score of GridSearchCV(cv=3,
             estimator=KerasClassifier(build_fn=<function create_model at 0x00000291EFFC3380>, layers=1, neurons=32),
             n_jobs=-1,
             param_grid={'epochs': [50, 100], 'layers': [1, 2, 3],
                         'neurons': [16, 32, 64, 128]})>